In [1]:
import dmnet.util as u
from snorkel.labeling import LFAnalysis, filter_unlabeled_dataframe
from snorkel.labeling.model import LabelModel, MajorityLabelVoter

In [2]:
desmat = u.pickle_load("gs://jdh-bucket/projects/snorkel_pos/dmatrics.dict")

In [3]:
mtx = u.pickle_load("gs://jdh-bucket/projects/snorkel_pos/lbl_matrix.mtx")

In [6]:
len(desmat['snorkel_tr'])

1000000

In [18]:
.0012 * 1000000

1200.0

In [19]:
100 - .12

99.88

In [11]:
mtx

array([[-1,  0, -1],
       [-1, -1, -1],
       [ 0,  0,  0],
       ...,
       [ 0, -1, -1],
       [-1, -1, -1],
       [ 0,  0, -1]])

In [20]:
ub = np.percentile(mtx, q=99.88, axis=0)
lb = np.percentile(mtx, q=60, axis=0)

In [21]:
mtx[mtx > ub] = 1
mtx[mtx <= lb] = 0
mtx[(mtx <= ub) & (mtx > lb)] = -1
mtx = mtx.astype(int)

In [22]:
LFAnalysis(mtx).lf_summary()

,Polarity,Coverage,Overlaps,Conflicts
0,"[0, 1]",1.0,1.0,0.002567
1,"[0, 1]",1.0,1.0,0.002567
2,"[0, 1]",1.0,1.0,0.002567


In [23]:
maj_mod = MajorityLabelVoter()
preds_train = maj_mod.predict(L=mtx)

In [14]:
maj_mod.score(L=mtx, Y=desmat['snorkel_tr']['y'], tie_break_policy="random")

{'accuracy': 0.856739}

In [15]:
lbl_mdl = LabelModel(cardinality=2, verbose=True)
lbl_mdl.fit(L_train=mtx, n_epochs=500, log_freq=100, seed=123)

INFO:root:Computing O...
INFO:root:Estimating \mu...
  0%|                                                | 0/500 [00:00<?, ?epoch/s]INFO:root:[0 epochs]: TRAIN:[loss=0.516]
INFO:root:[100 epochs]: TRAIN:[loss=0.000]
 60%|█████████████████████▍              | 298/500 [00:00<00:00, 1461.73epoch/s]INFO:root:[300 epochs]: TRAIN:[loss=0.000]
INFO:root:[400 epochs]: TRAIN:[loss=0.000]
100%|████████████████████████████████████| 500/500 [00:00<00:00, 1469.02epoch/s]
INFO:root:Finished Training


In [17]:
lbl_mdl.score(L=mtx, Y=desmat['snorkel_tr']['y'], tie_break_policy="random")

{'accuracy': 0.85664}